<h1 align="center" class="jp-toc-ignore" style="color: LightSeaGreen">Coder Mê Tài Chính</h1>

---
<table width="100%" style="border:0px;">
    <tr style="font-size: 14pt">
        <td><b>Evangelist:</b> Lê Minh Đạt</td>
        <td><b>Email:</b> <a href="mailto:sirminhdat@gmail.com">sirminhdat@gmail.com</a></td>
        <td><b>Zalo:</b> 0919 564 515</td>
    </tr>
</table>

<center><h1 class="jp-toc-ignore"><b>Python For Finance (Py4Fin)</b></h1></center>

<center>
    <h2 style="margin-bottom:0px;" class="jp-toc-ignore">BÀI 8</h2>
    <h1 style="color: Crimson; margin-top:10px; margin-bottom:0px">Mô Hình Định Giá Tài Sản Vốn</h1>
    <h1 style="color: Crimson; margin-top:10px;">Capital Asset Pricing Model (CAPM)</h1>
</center>

**Nội dung:**

1. CAPM là gì?
2. Công thức tính $\beta$ và ý nghĩa
3. Tính $\beta$
    - Load và xử lý dữ liệu chứng khoán
    - Tính $\beta$ theo công thức
    - Tính $\beta$ theo phương pháp hồi quy
      - Dùng hàm linregress của scipy.stats
      - Dùng hàm polyfit của numpy
4. Tính Expected Return bằng CAPM
    - Quy đổi RM và RF từ daily sang annualized
    - Tính Expected Return
    - Tính Return từ dữ liệu lịch sử
    - Tạo Dataframe so sánh dữ liệu
---


<h2 style="color: blue">1. CAPM là gì?</h2>
CAPM được phát triển bởi chuyên gia kinh tế tài chính William Sharpe vào những năm 60 của thế kỷ 20. CAPM mô tả mối liên hệ giữa lợi nhuận/TSSL kỳ vọng (Expected Return) và rủi ro hệ thống (Systematic Risk)/rủi ro thị trường (Market Risk)

$$
\Large \bar{R_i} = R_f + \beta_{i}(\bar{R}_m - R_f)
$$

- $\bar{R_i}:$ Lợi nhuận/TSSL kỳ vọng
- $R_f:$ Lãi suất phi rủi ro. Thường là lãi suất trái phiếu chính phủ
- $\beta_{i}:$ Mức độ biến động (rủi ro) của tài sản (thông thường là cổ phiếu/danh mục) so với thị trường
- $\bar{R}_m:$ Lợi nhuận/TSSL kỳ vọng của thị trường

**Ưu điểm:**

1. CAPM là một mô hình tương đối đơn giản, dễ hiểu. Nó chỉ yêu cầu một đầu vào là beta, để đo lường giá trị của tài sản biến động so với thị trường chung.
2. CAPM dựa trên nền tảng lý thuyết vững chắc và phù hợp với lý thuyết danh mục đầu tư hiện đại, đã được chấp nhận rộng rãi trong cộng đồng tài chính.
3. CAPM có thể được sử dụng để cung cấp mức lợi nhuận/TSSL yêu cầu chuẩn để so sánh lợi nhuận/TSSL kỳ vọng của từng tài sản.

**Nhược điểm:**

1. CAPM dựa trên một số giả định, chẳng hạn như thị trường hiệu quả giả thuyết và giả định của các nhà đầu tư hợp lý, có thể không luôn giữ trong thị trường thực tế.
2. CAPM chỉ xem xét một yếu tố là beta khi xác định lợi nhuận kỳ vọng của một tài sản. Điều này có thể không đủ để nắm bắt đầy đủ các đặc điểm rủi ro và lợi nhuận của tài sản.
3. CAPM được thiết kế chủ yếu để sử dụng với các tài sản được giao dịch công khai, chẳng hạn như cổ phiếu và có thể không phù hợp với các loại tài sản khác như: bất động sản hoặc vốn cổ phần tư nhân (Private Equity).

<h2 style="color: blue">2. Công thức tính $\beta$ và ý nghĩa</h2>

$$
\Large \beta_{i} = \frac{Cov(R_i,R_m)}{Var(R_m)}
$$

- $R_i:$ $(\bar{R_i} - R_f)$
- $R_m:$ $(\bar{R}_m - R_f)$

**Ý nghĩa:**

- $\beta < -1$: cổ phiếu biến động ngược chiều mạnh so với biến động của thị trường.
- $\beta = -1$: cổ phiếu biến động ngược chiều với biến động của thị trường theo tỉ lệ 1:1.
- $-1 < \beta < 0$: cổ phiếu biến động ngược chiều so với biến động của thị trường.
- $\beta = 0$: cổ phiếu hoàn toàn độc lập so với thị trường.
- $0 < \beta < 1$: cổ phiếu biến động cùng chiều so với biến động của thị trường.
- $\beta = 1$: cổ phiếu biến động giống với biến động của thị trường theo tỉ lệ 1:1.
- $\beta > 1$: cổ phiếu biến động cùng chiều mạnh so với biến động của thị trường.

<h2 style="color: blue">3. Tính $\beta$</h2>

In [40]:
import numpy as np
import pandas as pd
import pprint

<h3 style="text-align:center;">Từ Chối Trách Nhiệm (Disclaimer)</h3>
<h4 style="text-align:center; color: Crimson">Video này chỉ nhằm mục đích giáo dục, học thuật. Dữ liệu và thông tin trong video này không phải là lời khuyên đầu tư</h4>
<h4 style="text-align:center; color: Crimson">(This video is meant for educational purposes only. Data and information in this video is not an investment advice)</h4>

<h3 style="color: blue">3.1. Load và xử lý dữ liệu chứng khoán</h3>

In [41]:
df = pd.read_csv('data/stock/CAPM_data.csv')
df

,date,FRT,MSN,MWG,PNJ,REE,TCB,VNINDEX
0,2020-01-02,11.982,46.122,37.731,59.972,25.945,23.80,966.67
1,2020-01-03,12.478,46.527,37.183,59.561,25.629,23.65,965.14
2,2020-01-06,12.092,45.798,36.990,58.874,25.594,23.05,955.79
3,2020-01-07,12.037,45.960,37.280,59.629,25.735,23.20,958.88
4,2020-01-08,11.708,45.313,36.377,58.394,25.068,22.75,948.98
...,...,...,...,...,...,...,...,...
1010,2024-01-16,98.200,67.100,44.300,86.400,55.600,34.15,1163.12
1011,2024-01-17,99.500,66.300,44.900,88.600,56.100,34.05,1162.53
1012,2024-01-18,99.700,66.400,45.850,89.000,55.700,34.40,1169.06
1013,2024-01-19,99.900,67.800,46.050,88.200,56.000,34.95,1181.50


In [42]:
df['date'] = pd.to_datetime(df['date'])
df.set_index('date', inplace=True)
df

,FRT,MSN,MWG,PNJ,REE,TCB,VNINDEX
date,,,,,,,
2020-01-02,11.982,46.122,37.731,59.972,25.945,23.80,966.67
2020-01-03,12.478,46.527,37.183,59.561,25.629,23.65,965.14
2020-01-06,12.092,45.798,36.990,58.874,25.594,23.05,955.79
2020-01-07,12.037,45.960,37.280,59.629,25.735,23.20,958.88
2020-01-08,11.708,45.313,36.377,58.394,25.068,22.75,948.98
...,...,...,...,...,...,...,...
2024-01-16,98.200,67.100,44.300,86.400,55.600,34.15,1163.12
2024-01-17,99.500,66.300,44.900,88.600,56.100,34.05,1162.53
2024-01-18,99.700,66.400,45.850,89.000,55.700,34.40,1169.06


In [43]:
tickers = list(df.columns)
tickers

['FRT', 'MSN', 'MWG', 'PNJ', 'REE', 'TCB', 'VNINDEX']

In [44]:
daily_return_df = pd.DataFrame(index=df.index)

for col in df.columns:
    daily_return_df[col] = df[col].pct_change()
    
daily_return_df = daily_return_df.dropna()
daily_return_df

,FRT,MSN,MWG,PNJ,REE,TCB,VNINDEX
date,,,,,,,
2020-01-03,0.041395,0.008781,-0.014524,-0.006853,-0.012180,-0.006303,-0.001583
2020-01-06,-0.030934,-0.015668,-0.005191,-0.011534,-0.001366,-0.025370,-0.009688
2020-01-07,-0.004548,0.003537,0.007840,0.012824,0.005509,0.006508,0.003233
2020-01-08,-0.027332,-0.014077,-0.024222,-0.020711,-0.025918,-0.019397,-0.010325
2020-01-09,-0.032883,0.017854,0.015064,0.012929,0.000000,0.013187,0.011771
...,...,...,...,...,...,...,...
2024-01-16,0.013416,0.022866,0.032634,0.022485,0.007246,-0.004373,0.007798
2024-01-17,0.013238,-0.011923,0.013544,0.025463,0.008993,-0.002928,-0.000507
2024-01-18,0.002010,0.001508,0.021158,0.004515,-0.007130,0.010279,0.005617


In [45]:
bonds_df = pd.read_csv('data/stock/CAPM_vn_bonds_rate_3yrs.csv')
bonds_df

,date,close,open,high,low
0,22/01/2024,1.397,1.397,1.397,1.397
1,19/01/2024,1.447,1.447,1.447,1.447
2,18/01/2024,1.446,1.446,1.446,1.446
3,17/01/2024,1.443,1.443,1.443,1.443
4,16/01/2024,1.489,1.491,1.491,1.489
...,...,...,...,...,...
1053,07/01/2020,1.726,1.756,1.756,1.726
1054,06/01/2020,1.756,1.802,1.802,1.756
1055,03/01/2020,1.802,1.824,1.824,1.802
1056,02/01/2020,1.825,1.833,1.833,1.825


In [46]:
bonds_df['date'] = pd.to_datetime(bonds_df['date'], format="%d/%m/%Y") # dayfirst=True
bonds_df.set_index('date', inplace=True)
bonds_df.sort_index(inplace=True)
bonds_df = bonds_df[['close']].rename(columns={'close': 'annualized_3yrs_bonds'})
bonds_df

,annualized_3yrs_bonds
date,
2020-01-01,1.734
2020-01-02,1.825
2020-01-03,1.802
2020-01-06,1.756
2020-01-07,1.726
...,...
2024-01-16,1.489
2024-01-17,1.443
2024-01-18,1.446


In [47]:
bonds_df['annualized_3yrs_bonds'] = bonds_df['annualized_3yrs_bonds'] / 100
bonds_df

,annualized_3yrs_bonds
date,
2020-01-01,0.01734
2020-01-02,0.01825
2020-01-03,0.01802
2020-01-06,0.01756
2020-01-07,0.01726
...,...
2024-01-16,0.01489
2024-01-17,0.01443
2024-01-18,0.01446


$$
\Large R_{daily}=\biggl(1+R_{annualized}\biggr)^\frac{\displaystyle 1}{\displaystyle 365} - 1
$$

In [48]:
# bonds_df['daily_3yrs_bonds'] = (1 / (1 - bonds_df['annualized_3yrs_bonds'] * (3 * 365) / 365))**(1/(3 * 365)) - 1

# bonds_df['daily_3yrs_bonds'] =  np.power(1 + bonds_df['annualized_3yrs_bonds'], 1/365) - 1
bonds_df['daily_3yrs_bonds'] =  (1 + bonds_df['annualized_3yrs_bonds']) ** (1/365) - 1
bonds_df

,annualized_3yrs_bonds,daily_3yrs_bonds
date,,
2020-01-01,0.01734,0.000047
2020-01-02,0.01825,0.000050
2020-01-03,0.01802,0.000049
2020-01-06,0.01756,0.000048
2020-01-07,0.01726,0.000047
...,...,...
2024-01-16,0.01489,0.000040
2024-01-17,0.01443,0.000039
2024-01-18,0.01446,0.000039


In [49]:
daily_return_bonds_df = pd.merge(daily_return_df, bonds_df, left_index=True, right_index=True)
daily_return_bonds_df

,FRT,MSN,MWG,PNJ,REE,TCB,VNINDEX,annualized_3yrs_bonds,daily_3yrs_bonds
date,,,,,,,,,
2020-01-03,0.041395,0.008781,-0.014524,-0.006853,-0.012180,-0.006303,-0.001583,0.01802,0.000049
2020-01-06,-0.030934,-0.015668,-0.005191,-0.011534,-0.001366,-0.025370,-0.009688,0.01756,0.000048
2020-01-07,-0.004548,0.003537,0.007840,0.012824,0.005509,0.006508,0.003233,0.01726,0.000047
2020-01-08,-0.027332,-0.014077,-0.024222,-0.020711,-0.025918,-0.019397,-0.010325,0.01574,0.000043
2020-01-09,-0.032883,0.017854,0.015064,0.012929,0.000000,0.013187,0.011771,0.01601,0.000044
...,...,...,...,...,...,...,...,...,...
2024-01-16,0.013416,0.022866,0.032634,0.022485,0.007246,-0.004373,0.007798,0.01489,0.000040
2024-01-17,0.013238,-0.011923,0.013544,0.025463,0.008993,-0.002928,-0.000507,0.01443,0.000039
2024-01-18,0.002010,0.001508,0.021158,0.004515,-0.007130,0.010279,0.005617,0.01446,0.000039


In [50]:
RmRF_df = pd.DataFrame(index=daily_return_bonds_df.index)
for t in tickers:
    RmRF_df[f"{t}mRF"] = daily_return_bonds_df[t] - daily_return_bonds_df['daily_3yrs_bonds']
RmRF_df    

,FRTmRF,MSNmRF,MWGmRF,PNJmRF,REEmRF,TCBmRF,VNINDEXmRF
date,,,,,,,
2020-01-03,0.041346,0.008732,-0.014573,-0.006902,-0.012229,-0.006351,-0.001632
2020-01-06,-0.030982,-0.015716,-0.005238,-0.011582,-0.001413,-0.025418,-0.009735
2020-01-07,-0.004595,0.003490,0.007793,0.012777,0.005462,0.006461,0.003186
2020-01-08,-0.027375,-0.014120,-0.024265,-0.020754,-0.025961,-0.019439,-0.010367
2020-01-09,-0.032927,0.017810,0.015021,0.012886,-0.000044,0.013143,0.011727
...,...,...,...,...,...,...,...
2024-01-16,0.013375,0.022825,0.032594,0.022445,0.007206,-0.004414,0.007758
2024-01-17,0.013199,-0.011962,0.013505,0.025424,0.008954,-0.002968,-0.000547
2024-01-18,0.001971,0.001469,0.021119,0.004475,-0.007169,0.010240,0.005578


<h3 style="color: blue">3.2. Tính $\beta$ theo công thức</h3>

$$
\Large \beta_{i} = \frac{Cov(R_i,R_m)}{Var(R_m)}
$$


- $R_i:$ $(\bar{R_i} - R_f)$
- $R_m:$ $(\bar{R}_m - R_f)$

In [51]:
cov = RmRF_df.cov()
cov

,FRTmRF,MSNmRF,MWGmRF,PNJmRF,REEmRF,TCBmRF,VNINDEXmRF
FRTmRF,0.001038,0.000274,0.000394,0.000272,0.000285,0.000294,0.000237
MSNmRF,0.000274,0.000567,0.000262,0.000201,0.000188,0.000219,0.000186
MWGmRF,0.000394,0.000262,0.000567,0.000276,0.000253,0.000311,0.000222
PNJmRF,0.000272,0.000201,0.000276,0.000422,0.000211,0.000217,0.000176
REEmRF,0.000285,0.000188,0.000253,0.000211,0.000465,0.000201,0.000168
TCBmRF,0.000294,0.000219,0.000311,0.000217,0.000201,0.000516,0.000239
VNINDEXmRF,0.000237,0.000186,0.000222,0.000176,0.000168,0.000239,0.000184


In [52]:
cov['FRTmRF'][-1]

0.00023726441811692744

In [53]:
cov.loc['FRTmRF']['VNINDEXmRF'] # cov.loc['FRTmRF'][-1]

0.00023726441811692744

In [54]:
VNINDEXmRF_var = cov.loc['VNINDEXmRF']['VNINDEXmRF'] # cov.iloc[-1][-1]
VNINDEXmRF_var

0.00018380712383603314

In [57]:
VNINDEXmRF_var = RmRF_df['VNINDEXmRF'].var() #sigma^2 <=> Variance
VNINDEXmRF_var

0.00018380712383603314

In [56]:
tickers

['FRT', 'MSN', 'MWG', 'PNJ', 'REE', 'TCB', 'VNINDEX']

In [58]:
beta_by_formula = {}
for t in tickers:
    if t != 'VNINDEX':
        beta_by_formula[t] = cov[f"{t}mRF"][-1] / VNINDEXmRF_var

print('-'*20 + 'Beta' + '-'*20)
pprint.pprint(beta_by_formula)

--------------------Beta--------------------
{'FRT': 1.2908336367233588,
 'MSN': 1.0120414466680667,
 'MWG': 1.2091446603271117,
 'PNJ': 0.955704673078092,
 'REE': 0.9127899419292369,
 'TCB': 1.30041106087046}


<h3 style="color: blue">3.3. Tính $\beta$ theo phương pháp hồi quy</h3>

$$
\Large (\bar{R_i} - R_f) = \alpha_i  + \beta_{i}(\bar{R}_m - R_f)
$$

Hệ số alpha $(\alpha)$ là phần lợi suất vượt trội  của một khoản đầu tư sau khi điều chỉnh theo sự biến động liên quan đến thị trường và các biến động ngẫu nhiên. 
Chỉ số này thường được sử dụng để đánh giá hiệu quả đầu tư của một cổ phiếu, một danh mục đầu tư hay một quỹ đầu tư.

<h4 style="color: blue">3.3.1. Dùng hàm linregress của scipy.stats</h4>

In [ ]:
# Dùng thư viện scipy. pip install scipy
from scipy import stats

In [59]:
RmRF_df 

,FRTmRF,MSNmRF,MWGmRF,PNJmRF,REEmRF,TCBmRF,VNINDEXmRF
date,,,,,,,
2020-01-03,0.041346,0.008732,-0.014573,-0.006902,-0.012229,-0.006351,-0.001632
2020-01-06,-0.030982,-0.015716,-0.005238,-0.011582,-0.001413,-0.025418,-0.009735
2020-01-07,-0.004595,0.003490,0.007793,0.012777,0.005462,0.006461,0.003186
2020-01-08,-0.027375,-0.014120,-0.024265,-0.020754,-0.025961,-0.019439,-0.010367
2020-01-09,-0.032927,0.017810,0.015021,0.012886,-0.000044,0.013143,0.011727
...,...,...,...,...,...,...,...
2024-01-16,0.013375,0.022825,0.032594,0.022445,0.007206,-0.004414,0.007758
2024-01-17,0.013199,-0.011962,0.013505,0.025424,0.008954,-0.002968,-0.000547
2024-01-18,0.001971,0.001469,0.021119,0.004475,-0.007169,0.010240,0.005578


In [60]:
beta_by_scipy = {}
alpha_by_scipy = {}

X = RmRF_df['VNINDEXmRF'] # X = (Rm - Rf)
for t in tickers:
    if t != 'VNINDEX':
        Y = RmRF_df[f"{t}mRF"] # Y = (Ri - Rf)
        result1 = stats.linregress(X, Y)
        beta_by_scipy[t] = result1.slope # Beta
        alpha_by_scipy[t] = result1.intercept # Alpha

print('-'*20 + 'Beta' + '-'*20)
pprint.pprint(beta_by_scipy)

print('-'*20 + 'Alpha' + '-'*20)
pprint.pprint(alpha_by_scipy)


--------------------Beta--------------------
{'FRT': 1.2908336367233588,
 'MSN': 1.0120414466680667,
 'MWG': 1.2091446603271117,
 'PNJ': 0.9557046730780921,
 'REE': 0.912789941929237,
 'TCB': 1.3004110608704602}
--------------------Alpha--------------------
{'FRT': 0.0022588261556253813,
 'MSN': 0.00036686390995083055,
 'MWG': 0.0001414408576526503,
 'PNJ': 0.0003049679984301055,
 'REE': 0.0007224532819241938,
 'TCB': 0.0002739969817456273}


<h4 style="color: blue">3.3.2. Dùng hàm polyfit của numpy</h4>

In [61]:
# Dùng thư viện Numpy
beta_by_np = {}
alpha_by_np = {}

X = RmRF_df['VNINDEXmRF'] # X = (Rm - Rf)
for t in tickers:
    if t != 'VNINDEX':
        Y = RmRF_df[f"{t}mRF"] # Y = (R - Rf)
        # result2 = np.polyfit(X, Y, deg=1) # deg=1 nghĩa là phương trình hồi quy là phương trình bậc I
        # beta_by_np[t] = result2[0] # Beta
        # alpha_by_np[t] = result2[1] # Alpha
        
        _beta, _alpha = np.polyfit(X, Y, deg=1) # deg=1 nghĩa là phương trình hồi quy là phương trình bậc I
        beta_by_np[t] = _beta
        alpha_by_np[t] = _alpha

print('-'*20 + 'Beta' + '-'*20)
pprint.pprint(beta_by_np)

print('-'*20 + 'Alpha' + '-'*20)
pprint.pprint(alpha_by_np)

--------------------Beta--------------------
{'FRT': 1.2908336367233586,
 'MSN': 1.0120414466680663,
 'MWG': 1.2091446603271108,
 'PNJ': 0.955704673078091,
 'REE': 0.9127899419292366,
 'TCB': 1.3004110608704589}
--------------------Alpha--------------------
{'FRT': 0.002258826155625382,
 'MSN': 0.0003668639099508306,
 'MWG': 0.00014144085765264983,
 'PNJ': 0.00030496799843010537,
 'REE': 0.000722453281924194,
 'TCB': 0.0002739969817456276}


In [62]:
beta_data = {
    'formula': beta_by_formula,
    'scipy': beta_by_scipy,
    'numpy': beta_by_np
}

beta_df = pd.DataFrame(data=beta_data)
beta_df

,formula,scipy,numpy
FRT,1.290834,1.290834,1.290834
MSN,1.012041,1.012041,1.012041
MWG,1.209145,1.209145,1.209145
PNJ,0.955705,0.955705,0.955705
REE,0.912790,0.912790,0.912790
TCB,1.300411,1.300411,1.300411


<h2 style="color: blue">4. Tính Expected Return bằng CAPM</h2>

$$
\Large \bar{R_i} = R_f + \beta_{i}(\bar{R}_m - R_f)
$$

In [64]:
daily_return_bonds_df

,FRT,MSN,MWG,PNJ,REE,TCB,VNINDEX,annualized_3yrs_bonds,daily_3yrs_bonds
date,,,,,,,,,
2020-01-03,0.041395,0.008781,-0.014524,-0.006853,-0.012180,-0.006303,-0.001583,0.01802,0.000049
2020-01-06,-0.030934,-0.015668,-0.005191,-0.011534,-0.001366,-0.025370,-0.009688,0.01756,0.000048
2020-01-07,-0.004548,0.003537,0.007840,0.012824,0.005509,0.006508,0.003233,0.01726,0.000047
2020-01-08,-0.027332,-0.014077,-0.024222,-0.020711,-0.025918,-0.019397,-0.010325,0.01574,0.000043
2020-01-09,-0.032883,0.017854,0.015064,0.012929,0.000000,0.013187,0.011771,0.01601,0.000044
...,...,...,...,...,...,...,...,...,...
2024-01-16,0.013416,0.022866,0.032634,0.022485,0.007246,-0.004373,0.007798,0.01489,0.000040
2024-01-17,0.013238,-0.011923,0.013544,0.025463,0.008993,-0.002928,-0.000507,0.01443,0.000039
2024-01-18,0.002010,0.001508,0.021158,0.004515,-0.007130,0.010279,0.005617,0.01446,0.000039


<h3 style="color: blue">4.1. Quy đổi RM và RF từ daily sang annualized</h3>

In [65]:
yearly_returns = {}
m = daily_return_bonds_df.shape[0]
m

1014

In [66]:
yearly_returns['RM'] = {
    'cum': np.power((1 + daily_return_bonds_df['VNINDEX']).prod(), 365/m) - 1,
    'avg': daily_return_bonds_df['VNINDEX'].mean() * 252
}

In [67]:
yearly_returns['RF'] = {
    'cum': np.power((1 + daily_return_bonds_df['daily_3yrs_bonds']).prod(), 365/m) - 1,
    'avg': daily_return_bonds_df['daily_3yrs_bonds'].mean() * 252
}

In [68]:
pprint.pprint(yearly_returns)

{'RF': {'avg': 0.013042911156474648, 'cum': 0.019070377326379395},
 'RM': {'avg': 0.073492559222334, 'cum': 0.07535629070951799}}


In [69]:
daily_return_bonds_df['annualized_3yrs_bonds'].mean()

0.019149694280078896

<h3 style="color: blue">4.2. Tính Expected Return</h3>

In [70]:
CAPM = {
    'cum': {},
    'avg': {}
}

for _ticker, _beta in beta_by_formula.items():
    CAPM['cum'][_ticker] = round(yearly_returns['RF']['cum'] + _beta * (yearly_returns['RM']['cum'] - yearly_returns['RF']['cum']), 4)
    CAPM['avg'][_ticker] = round(yearly_returns['RF']['avg'] + _beta * (yearly_returns['RM']['avg'] - yearly_returns['RF']['avg']), 4)

pprint.pprint(CAPM)

{'avg': {'FRT': 0.0911,
         'MSN': 0.0742,
         'MWG': 0.0861,
         'PNJ': 0.0708,
         'REE': 0.0682,
         'TCB': 0.0917},
 'cum': {'FRT': 0.0917,
         'MSN': 0.076,
         'MWG': 0.0871,
         'PNJ': 0.0729,
         'REE': 0.0704,
         'TCB': 0.0923}}


<h3 style="color: blue">4.3. Tính Return từ dữ liệu lịch sử</h3>

In [71]:
daily_return_bonds_df

,FRT,MSN,MWG,PNJ,REE,TCB,VNINDEX,annualized_3yrs_bonds,daily_3yrs_bonds
date,,,,,,,,,
2020-01-03,0.041395,0.008781,-0.014524,-0.006853,-0.012180,-0.006303,-0.001583,0.01802,0.000049
2020-01-06,-0.030934,-0.015668,-0.005191,-0.011534,-0.001366,-0.025370,-0.009688,0.01756,0.000048
2020-01-07,-0.004548,0.003537,0.007840,0.012824,0.005509,0.006508,0.003233,0.01726,0.000047
2020-01-08,-0.027332,-0.014077,-0.024222,-0.020711,-0.025918,-0.019397,-0.010325,0.01574,0.000043
2020-01-09,-0.032883,0.017854,0.015064,0.012929,0.000000,0.013187,0.011771,0.01601,0.000044
...,...,...,...,...,...,...,...,...,...
2024-01-16,0.013416,0.022866,0.032634,0.022485,0.007246,-0.004373,0.007798,0.01489,0.000040
2024-01-17,0.013238,-0.011923,0.013544,0.025463,0.008993,-0.002928,-0.000507,0.01443,0.000039
2024-01-18,0.002010,0.001508,0.021158,0.004515,-0.007130,0.010279,0.005617,0.01446,0.000039


In [72]:
tickers[:-1]

['FRT', 'MSN', 'MWG', 'PNJ', 'REE', 'TCB']

In [73]:
avg_historical = round(daily_return_bonds_df[tickers[:-1]].mean() * 252, 4)
avg_historical

FRT    0.6603
MSN    0.1667
MWG    0.1218
PNJ    0.1477
REE    0.2503
TCB    0.1607
dtype: float64

In [74]:
cum_historical = round(np.power((1 + daily_return_bonds_df[tickers[:-1]]).prod(), 365/m) - 1, 4)
cum_historical

FRT    1.1525
MSN    0.1481
MWG    0.0752
PNJ    0.1466
REE    0.3199
TCB    0.1483
dtype: float64

<h3 style="color: blue">4.4. Tạo Dataframe so sánh dữ liệu</h3>

In [77]:
# Tạo DataFrame so sánh Return tính bằng CAPM và Return tính từ dữ liệu lịch sử
return_data = {
    'Mean_CAPM': CAPM['avg'],
    'Mean_Historical': avg_historical,
    'Cumulative_CAPM': CAPM['cum'],
    'Cumulative_Historical': cum_historical
}

compare_df = pd.DataFrame(data=return_data)
compare_df.columns = pd.MultiIndex.from_tuples([col.split('_') for col in compare_df.columns])
compare_df

Mean            Cumulative           
       CAPM Historical       CAPM Historical
FRT  0.0911     0.6603     0.0917     1.1525
MSN  0.0742     0.1667     0.0760     0.1481
MWG  0.0861     0.1218     0.0871     0.0752
PNJ  0.0708     0.1477     0.0729     0.1466
REE  0.0682     0.2503     0.0704     0.3199
TCB  0.0917     0.1607     0.0923     0.1483